In [4]:
import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [5]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
df.drop(columns=['day'])
# display the table



,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
29/02/20,105
01/03/20,140
02/03/20,166
03/03/20,229
04/03/20,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [6]:
fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases')
fig.show()

[0.65       0.45598592 1.        ]
`ftol` termination condition is satisfied.
Function evaluations 15, initial cost 1.0515e+01, final cost 3.6675e-03, first-order optimality 1.56e-08.
            Exponential model
            x0 = -24.114323080337044
            y0 = -560.445954682993
            k = 0.07751803140471744
            
`gtol` termination condition is satisfied.
Function evaluations 24, initial cost 2.7441e+00, final cost 2.2560e-04, first-order optimality 8.66e-10.
        Sigmoid model
        x0 = 77.55712208149683
        y0 = -89.37758752972191
        c = 4543.244697315732
        k = 0.17417924523529288
        asymptot = 4453.867109786011
        flex = 77.55712208149683, 2182.244761128144
        
`ftol` termination condition is satisfied.
Function evaluations 38, initial cost 1.0828e+00, final cost 2.7773e-04, first-order optimality 1.10e-08.
        Sigmoid derivative model
        x0 = 86.04748122343696
        y0 = -127.03486842026453
        c = 89733.8391232